# Trees.org
https://trees.org

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
with open('../input/TreesOrg/data.json', 'r') as file:
    data = json.load(file)
type(data)

In [ ]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
df.info()

In [ ]:
df['country'] = df['address'].astype(str).apply(lambda x: x.split()[-1] if isinstance(x, str) else '')
df.head(1)

In [ ]:
columns_rename_mapping = {
    'ID': 'project_id_reported',
    'coordinates': 'geometry',
    'info': 'project_description_reported',
    'country': 'country',
    'date': 'planting_date_reported',
    'link': 'url'
}
df = df[list(columns_rename_mapping.keys())]
df.rename(columns=columns_rename_mapping, inplace=True)

### Fix geometries

In [ ]:
df[['lon', 'lat']] = df['geometry'].str.split(',', expand=True).astype(float)
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
df.drop(columns = ['lon', 'lat'], inplace = True)

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

### Harmonize nomenclature

In [ ]:
gdf['planting_date_reported'] = pd.to_datetime(gdf['planting_date_reported']).dt.year

In [ ]:
gdf['host_name'] = 'trees.org'
gdf = gdf.assign(site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,site_id_reported=None)
gdf.info()

In [ ]:
gdf.to_file('../midsave/trees_org.gpkg')